### Importing necessary libs

In [1]:
import os
from nltk import tokenize
import pandas as pd
from sentence_transformers import SentenceTransformer
import re
from Utility.ThesisUtility import mbkmeans_clusters,writeClustersToExcel

###  Constants

In [ ]:
dataFilePath = r"D:\UDE\6th Semester\MEMS\MEWS Data\MEWS_Essays\MEWS_Essays\Essays_all\Only 100"
k = 19
modelFileName = "SbertSentences.txt"
clusterExcelSheetName = "clusteredArgument- Sbert - "
#defining the model
model = SentenceTransformer('all-mpnet-base-v2', device="cuda")

### Structring words to use them for splitting the sentences

In [7]:
listStructureWords =  []

dfSrtucturingWords = pd.read_excel("../CSV Files/structureWords.xlsx")
listStructureWords = dfSrtucturingWords.iloc[:,0].tolist()

### Reading the essays from the directory

In [ ]:
regex = r"\b(?:{})\b".format("|".join(listStructureWords))
sentences = []


for filename in os.listdir(dataFilePath):
   with open(os.path.join(dataFilePath, filename)) as f:
       text = f.read()
       text = text.replace("ï»¿","")
       sents = re.split(regex, text)
       sents = tokenize.sent_tokenize(text)
       for s in sents:
           sentses = tokenize.sent_tokenize(s)
           if (s.isspace() or len(s) ==0):
               continue
           s = s.lower()
           if (len(s.split()) <5):
               continue
           for ss in sentses:
                 if (len(ss.split()) >5):
                     sentences.append(ss)

### Transorming the text of the sentences into vectors

In [ ]:
corpus_embeddings = model.encode(sentences, show_progress_bar =True, device="cuda")
# corpus_embeddings.shape

### applying the clustering method

In [ ]:
clustering, cluster_labels = mbkmeans_clusters(X=corpus_embeddings, k=k, print_silhouette_values=True)

writeClustersToExcel(clusterExcelSheetName,cluster_labels,k,sentences,corpus_embeddings,clustering)
